# α-AGI MARK Bonding Curve Validation
This notebook mirrors the arithmetic-series pricing formulas implemented in `AlphaMarkEToken.sol`.
It cross-checks the closed-form equations against discrete summation for a range of scenarios.

In [ ]:
from decimal import Decimal, getcontext
getcontext().prec = 50

def purchase_cost(base_price, slope, supply, amount):
    base_component = base_price * amount
    slope_component = slope * (amount * ((2 * supply) + amount - 1) / 2)
    return base_component + slope_component

def sale_return(base_price, slope, supply, amount):
    base_component = base_price * amount
    if amount == 0:
        return base_component
    slope_component = slope * (amount * ((2 * (supply - 1)) - (amount - 1)) / 2)
    return base_component + slope_component

def discrete_purchase(base_price, slope, supply, amount):
    return sum(base_price + slope * (supply + i) for i in range(amount))

def discrete_sale(base_price, slope, supply, amount):
    return sum(base_price + slope * (supply - 1 - i) for i in range(amount))

scenarios = [
    (Decimal('0.1'), Decimal('0.05'), 0, 3, 2),
    (Decimal('0.25'), Decimal('0.04'), 5, 4, 3),
    (Decimal('0.5'), Decimal('0.02'), 10, 1, 5),
]

for base, slope, supply, purchase_amount, sale_amount in scenarios:
    purchase_closed = purchase_cost(base, slope, supply, purchase_amount)
    purchase_discrete = Decimal(discrete_purchase(base, slope, supply, purchase_amount))
    sale_closed = sale_return(base, slope, supply + purchase_amount, sale_amount)
    sale_discrete = Decimal(discrete_sale(base, slope, supply + purchase_amount, sale_amount))
    print('Scenario:', base, slope, supply, purchase_amount, sale_amount)
    print('  purchase closed   =', purchase_closed)
    print('  purchase discrete =', purchase_discrete)
    print('  sale closed       =', sale_closed)
    print('  sale discrete     =', sale_discrete)
    assert purchase_closed == purchase_discrete
    assert sale_closed == sale_discrete
print('All scenarios consistent.')
